## This script compares age metrics among models that use uniform versus heterogeneous porosity.  The goal is to evaluate if and how complexity influences age metrics across scales.

#### The script reads in the model_grid.csv file for each model and uses that to tie each particle's initial location to select categories, such as the HUC, NLCD, coarse fraction, etc.  The model_grid.csv file was created via verion 2 of the general models / GRTD notebooks, tied to PFJ's repo called "GenMod" on the USGS GitLab site.

In [ ]:
__author__ = 'Paul Juckem'
%matplotlib notebook
import os, sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import matplotlib.cm as cm
from datetime import datetime
import gdal
from gdal import ogr, osr
import gen_mod_functions as gm
import flopy as fp
import pickle
from ipywidgets import interact, Dropdown, Text
from IPython.display import display
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import csv

import fit_parametric_distributions

try:
    import rasterio
except:
    print('Install rasterio into your python environment. Or, skip plotting of geotiffs at the end of this notebook')

modifier = False
def ReturnEntry(sender):
    modifier.value = intext.value

## Specify user input, including list of models to compare and which axes to plot them, plus attributes to analyze.

In [ ]:
#simulate_list = ['FWP1L_zK', 'FWP5L_zK', 'FWP5L_hK']  # list of models (w/ matching directory name) to analyze
#compare_list = ('FWP5L_zK', 'FWP5L_hK')  # compare these models directly; both uni_n and het_n versions.  Use a tuple to preserve order
evaluate = ('FWP5L_hK', 'FWP5L_zK')  # which two models to compare with color flooding of model factors.  1st minus 2nd, so order matters!
abrevdict = {'FWP5L_zK':'5zK', 'FWP5L_hK':'5hK'}

# This variable facilitates scenario testing (stopping or not at weak sinks/sources; porosity values & configuration, etc.)
#scenario_name = None  # set to None if no additional text was added to the modpath file names.
#scenario_name = 'passthroughsnk'  # an optional text string added to MP file names. MUST match value used in NB 01aa!!!
scenario_name = 'het_n'  

# when comparing results of the scenario to the base case, which weighting option to use? 'flux' or 'volume'
base_case = 'flux'

# for plotting 1:1 graphs
#x1, y1 = 0, 1  # index of names in the yet-to-be-populated eptlist variable.  (0=base case; 1 = scenario)

In [ ]:
surf_aq_lays = 3  # deepest layer of the surficial aquifer.
nrow, ncol = 930, 650  # easier to hardcode this than waste the time to read-in a model just to get DIS info.

# for labeling RTD plots
label_by_model_name = True  # if false, will include zone info in label.

category = 'HUC8' 
age_cutoff = 65
minptl = 500  # The minimum number of particles for EACH model within each HUC (eg: if FWP5L has 2000, 
               # but FWP1L has 800, none get plotted in the 1:1 plots.  Still included in the RTD plots.)
               # 500-1000 seems reasonable for 1:1 plots as it includes only HUCs with really refined RTDs;
               # however, 100 - 500 seems more reasonable if we want to visualize spatial patterns because it
               # allows more HUCs to be plotted.  100-500 is based on visually inspecting RTD curves.
                
# Columns in the model_grid.csv file to keep.  Purge all others.
mg_columns = ['node_num', 'row', 'col', 'HUC6', 'HUC8', 'HUC10', 'HUC12', 'ibound', 'gage_id', 'coarse_flag', 
              'qu_atlas', 'catchment', 'ssurgo', 'stream_order', 'surfmat']
                
purge_hucs = [40602, 4060200, 406020000, 40602000000]  # all hucs for Lake Michigan      

HUCshpdict = {'HUC6':'E:/HUCS/WBD_4n7/WBD_HUC6_UTMft_FWPdomain.shp', 
              'HUC8':'E:/HUCS/WBD_4n7/WBD_HUC8_UTMft_FWPdomain.shp', 
              'HUC10':'E:/HUCS/WBD_4n7/WBD_HUC10_UTMft_FWPdomain.shp', 
              'HUC12':'E:/HUCS/WBD_4n7/WBD_HUC12_UTMft_FWPdomain.shp'}

HUCtiffdict = {'HUC6':'E:/HUCS/WBD_4n7/HUC6_UTMft_FWP.tiff', 
              'HUC8':'E:/HUCS/WBD_4n7/HUC8_UTMft_FWP.tiff', 
              'HUC10':'E:/HUCS/WBD_4n7/HUC10_UTMft_FWP.tiff',
              'HUC12':'E:/HUCS/WBD_4n7/HUC12_UTMft_FWP.tiff'}

dischargerastdict = {'FWP1L_zK': '../TIFFs/FWP1L_zK_SFRgain.tif', 'FWP5L_zK':'../TIFFs/FWP5L_zK_SFRgain.tif', 'FWP5L_hK':
                     '../TIFFs/FWP5L_hK_SFRgain.tif'}
lossrastdict = {'FWP1L_zK': '../TIFFs/FWP1L_zK_SFRloss.tif', 'FWP5L_zK':'../TIFFs/FWP5L_zK_SFRloss.tif', 'FWP5L_hK':
                     '../TIFFs/FWP5L_hK_SFRloss.tif'}

#### Prep the script for the models to be analyzed

In [ ]:
homes = ['../Models']
fig_dir = '../Figures'

if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)

mfpth = '../executables/MODFLOW-NWT_1.0.9/bin/MODFLOW-NWT_64.exe'
mp_exe_name = '../executables/modpath.6_0/bin/mp6x64.exe' 

dir_list = []
modlist = []
i = 0
r = 0

path_dict = {}
dfdict = {}
totp = {}
for home in homes:
    if os.path.exists(home):
        for dirpath, dirnames, filenames in os.walk(home):
            for f in filenames:
                if os.path.splitext(f)[-1] == '.nam':
                    mod = os.path.splitext(f)[0]
                    i += 1
                    #if mod in compare_list:
                    if mod in evaluate:
                        modlist.append(mod)
                        dir_list.append(dirpath)
                        r += 1
                        path_dict[mod] = dirpath
                               
print('    {} models read'.format(i))
print('These {} models will be analyzed: {}'.format(r, modlist))

In [ ]:
# generate list of nam files:
nam_list = []
for pth in dir_list:
    model = os.path.normpath(pth).split(os.sep)[2]
    nam_file = '{}.nam'.format(model)
    nam_list.append(nam_file)

In [ ]:
# setup dictionaries of the MODFLOW units for proper labeling of figures.
lenunit = {0:'undefined units', 1:'feet', 2:'meters', 3:'centimeters'}
timeunit = {0:'undefined', 1:'second', 2:'minute', 3:'hour', 4:'day', 5:'year'}

# Create dictionary of multipliers for converting model time units to days
time_dict = dict()
time_dict[0] = 1.0 # undefined assumes days, so enter conversion to days
time_dict[1] = 24 * 60 * 60
time_dict[2] = 24 * 60
time_dict[3] = 24
time_dict[4] = 1.0
time_dict[5] = 1.0

In [ ]:
# Read-in model info and check max/min nlay & create list of DIS objects.  
# Assumes all else is the same among models (hnoflow, hdry, etc)
print ('Reading model information')
nlay_min = 100
nlay_max = 0

dis_objs = []
for i, model in enumerate(nam_list):
    nam_file = model
    model_ws = dir_list[i]
    
    fpmg = fp.modflow.Modflow.load(nam_file, model_ws=model_ws, exe_name=mfpth, version='mfnwt', 
                                   load_only=['DIS', 'BAS6', 'UPW', 'OC'], check=False)

    dis = fpmg.get_package('DIS')
    dis_objs.append(dis)
    bas = fpmg.get_package('BAS6')
    upw = fpmg.get_package('UPW')
    oc = fpmg.get_package('OC')

    delr = dis.delr
    delc = dis.delc
    nlay = dis.nlay
    nrow = dis.nrow
    ncol = dis.ncol
    bot = dis.getbotm()
    top = dis.gettop()

    hnoflo = bas.hnoflo
    ibound = np.asarray(bas.ibound.get_value())
    hdry = upw.hdry
    
    if nlay > nlay_max:
        nlay_max = nlay
    if nlay < nlay_min:
        nlay_min = nlay
        
    print('  .. done reading model {}'.format(i+1))

print ('   ... all done') 

print('minimum layers in a model:  {}'.format(nlay_min))
print('maximum layers in a model:  {}'.format(nlay_max))

## Process endpoint files and merge with model_grid.csv

In [ ]:
def purge(ep_data):
    pre_Quaternary = ep_data.loc[ep_data.rt>=2.6e6]
    pre_Cretaceous = ep_data.loc[ep_data.rt>=66e6]
    preCambrian = ep_data.loc[ep_data.rt>=541e6]
    pre_earth = ep_data.loc[ep_data.rt>=4.6e9]

    print('\nFor your information:')
    print('{} particles were simulated as being older than Earth!'.format(preCambrian.shape[0]))
    print('{} particles were simulated as being PreCambrian in age.'.format(preCambrian.shape[0]))
    print('{} particles were simulated as being Cretaceous in age or older.'.format(pre_Cretaceous.shape[0]))
    print('{} particles were simulated as being pre-Quaternary in age.'.format(pre_Quaternary.shape[0]))
    
    ep_data = ep_data.loc[ep_data.rt<4.6e9]
    print('Purged particles older than earth')
    return(ep_data)

In [ ]:
# read-in the model_grid.csv file for each model.  Then create a dataframe from each csv & pickle file.

dfdict = {}
totp = {}

for i, model in enumerate(evaluate):
#for i, model in enumerate(compare_list):
    g = os.path.join(path_dict[model], 'model_grid.csv')
    z = os.path.join(path_dict[model], 'zone_df.csv')
    try:
        df = pd.read_csv(g)
        df.ibound.replace(0, np.nan, inplace=True)
        df = df[df.ibound.notnull()]
        df = df[mg_columns]  # keep just the desired fields
        # re-calculate 2D cell number b/c node_num is computed differently for model_grid.csv and what's in the ept file.
        df['cellnum2d'] = df.row * ncol + df.col
        
        zone_df = pd.read_csv(z, index_col=0)
        for group in zone_df:
            print('\nReading-in the EPT file for {} in {}'.format(group, model))
            if scenario_name == None:
                print("Oops, I didn't really plan for this.  Stopping")
                sys.exit()
            else:
                p1 = os.path.join(path_dict[model], '{}_{}_{}.mpend'.format(model, base_case, group))
                p2 = os.path.join(path_dict[model], '{}_{}_{}_{}.mpend'.format(model, base_case, group, scenario_name))
                mnz1 = '{}_{}'.format(model, group)
                mnz2 = '{}_{}_{}'.format(model, group, scenario_name)
                eptu1 = fit_parametric_distributions.read_endpoints(p1, dis, time_dict)
                eptu2 = fit_parametric_distributions.read_endpoints(p2, dis, time_dict)
                
            eptu1 = purge(eptu1)
            eptu2 = purge(eptu2)
            eptu1['cellnum2d'] = (eptu1['Initial Row']-1) * ncol + (eptu1['Initial Column'] -1)  # -1 to convert to 0-based
            eptu2['cellnum2d'] = (eptu2['Initial Row']-1) * ncol + (eptu2['Initial Column'] -1)  # -1 to convert to 0-based
            eptu1_mg = eptu1.join(df, on='cellnum2d', lsuffix='_ept', rsuffix='_mg')
            eptu2_mg = eptu2.join(df, on='cellnum2d', lsuffix='_ept', rsuffix='_mg')
            eptu1_mg = eptu1_mg[eptu1_mg['Initial Layer'] <= surf_aq_lays]  # ensure that we're only analyzing Glacial!
            eptu2_mg = eptu2_mg[eptu2_mg['Initial Layer'] <= surf_aq_lays]  # ensure that we're only analyzing Glacial!

            #dfdict[model] = eptu_mg
            dfdict[mnz1] = eptu1_mg
            dfdict[mnz2] = eptu2_mg
            totp[mnz1] = eptu1_mg.rt.count()
            totp[mnz2] = eptu2_mg.rt.count()
            
            #x1 = eptlist[x1]  # pull out the file name after supplying the index
            #y1 = eptlist[y1]
            
    except (AttributeError, ValueError, IOError, IndexError):
        print('ERROR. THIS CODE BLOCK DID NOT COMPLETE. TROUBLE-SHOOT AND TRY AGAIN')
        print('The error occured while working on this model: {}'.format(model))
        raise SystemExit()

eptlist = list(dfdict.keys())
print('....done')

In [ ]:
print(eptlist)
print(modlist)  # can't control the order of this one
print(evaluate)

In [ ]:
# Plot age distributions for uniform and heterogeneous porosity models by selected HUC

uniques = dfdict[eptlist[0]][category].unique()
# remove any HUCs listed in purge_hucs
for h in purge_hucs:
    ind = np.where(uniques==h)
    uniques = np.delete(uniques, ind)

n_uni = len(uniques)
sum_p = {}
for mn in eptlist:
    sum_p[mn] = 0
    
if n_uni <= 20:
    vplots = int(np.ceil(n_uni/ 3.0))
    figsize = (12, 3*vplots)
    CS, CSaxes = plt.subplots(vplots, 3, figsize=figsize)
else:
    hplots = int(np.round(np.sqrt(n_uni)))
    vplots = int(np.ceil(np.sqrt(n_uni)))
    figsize = (hplots*4, hplots*3)
    CS, CSaxes = plt.subplots(vplots, hplots, figsize=figsize)
        
colors_line = plt.cm.brg(np.linspace(0, 1, len(eptlist)))

for ax, cat_val in zip(CSaxes.flat, uniques):
    n = []
    for i, md in enumerate(eptlist):
        rt = dfdict[md].loc[dfdict[md][category]==cat_val, 'rt']  # 'rt' is "raw time" in the dataframe
        rt.sort_values(inplace=True)
        n.append(rt.count())
        sum_p[md] = sum_p[md] + rt.count()
        y_rt = np.linspace(0, 1, rt.shape[0])
    
        if label_by_model_name:
            label = eptlist[i]
        else:
            label = md
        ax.plot(rt, y_rt, c=colors_line[i], label=label)
        ax.plot((age_cutoff, age_cutoff), (0.2, 1), 'k--')
        
        title = '{}: {}'.format(category, cat_val)
        ax.set_title(title, fontsize=12)

        ax.set_xscale('log')
        ax.set_xlim(1e0, 1e3)
        ax.set_ylim(0, )

        ax.legend(loc=0, frameon=False, fontsize=8)#, bbox_to_anchor=(0.20, 0.2), ncol=1)
        ax.set_xlabel('Residence time, in years')
        ax.set_ylabel('Cumulative frequency')
        if len(n) == len(modlist):
            nmin, nmax = min(n), max(n)
            ax.text(5,0.02, '# particles: {:,} - {:,}'.format(nmin, nmax))
        
CS.suptitle('Comparison of glacial particle time distributions by {} for FWP scenarios'.format(category), fontsize=18)  
CS.tight_layout()

if n_uni < 18:
    #CS.subplots_adjust(top= 0.86, hspace=0.85)
    CS.subplots_adjust(top= 0.86)
elif n_uni < 100:
    CS.subplots_adjust(top= 0.93, hspace=0.55)
elif n_uni < 400:
    CS.subplots_adjust(top= 0.95, hspace=0.55)
else:
    CS.subplots_adjust(top= 0.97, hspace=0.55)

#dst = 'RTD_compare--{}_{}'.format(category, scenario_name)
#dst_pth = os.path.join(fig_dir, dst)
#plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of Young Fraction for uniform (X) versus heterogeneous (Y) porosity models by selected HUC

x1, y1, x2, y2 = eptlist[2], eptlist[3], eptlist[0], eptlist[1]  # columns in eptlist. In this case (x=uniform n; y = hetero n)

uniques = dfdict[eptlist[0]][category].unique()
# remove any HUCs listed in purge_hucs
for h in purge_hucs:
    ind = np.where(uniques==h)
    uniques = np.delete(uniques, ind)

yfhucdict = {}
yfvalues = []
skiphuc = []
for j, cat_val in enumerate(uniques):  # each HUC ID
    yfmoddict = {}
    for i, md in enumerate(eptlist):  # each of the 3 FWP models
        mn = eptlist[i]
        df = dfdict[md][dfdict[md][category]==cat_val].copy()
        if df.rt.count() >= minptl:
            youngdf = df.loc[df.rt < age_cutoff]
            yf = youngdf.rt.count() / df.rt.count()
            yfmoddict[mn] = yf
            yfvalues.append(yf)
        else:
            skiphuc.append(cat_val)
            break
    if cat_val not in skiphuc:
        yfhucdict[cat_val] = yfmoddict    
        
ddd = pd.DataFrame(yfhucdict).T
Prho_y1, Pp_y1 = stats.pearsonr(np.array(ddd[x1]), np.array(ddd[y1]))
Srho_y1, Sp_y1 = stats.spearmanr(np.array(ddd[x1]), np.array(ddd[y1]))
std_y1 = stats.tstd(np.array(ddd[x1]) - np.array(ddd[y1]))
ME_y1 = np.mean(np.array(ddd[x1]) - np.array(ddd[y1]))
Prho_y2, Pp_y2 = stats.pearsonr(np.array(ddd[x2]), np.array(ddd[y2]))
Srho_y2, Sp_y2 = stats.spearmanr(np.array(ddd[x2]), np.array(ddd[y2]))
std_y2 = stats.tstd(np.array(ddd[x2]) - np.array(ddd[y2]))
ME_y2 = np.mean(np.array(ddd[x2]) - np.array(ddd[y2]))

#plotting
mini = min(min(ddd[x1]), min(ddd[x2]), min(ddd[y1]), min(ddd[y2]))
maxi = max(max(ddd[x1]), max(ddd[x2]), max(ddd[y1]), max(ddd[y2]))
yrange = maxi-mini
ax = ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='{}: uniform n vs heterogeneous n'.format(evaluate[1]))
ddd.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', label = '{}: uniform n vs heterogeneous n'.format(evaluate[0]), ax=ax)
lowlim = min(ax.get_xlim()[0], ax.get_ylim()[0])
uplim = max(ax.get_xlim()[1], ax.get_ylim()[1])
lims = (lowlim, uplim)
lrange = uplim - lowlim
ax.set_xlim(lims)  # force the plots to be square
ax.set_ylim(lims)
ax.plot((mini, maxi), (mini, maxi), 'k--')

plt.xlabel('Young Fraction from the uniform porosity models')
plt.ylabel('Young Fraction from the heterogeneous porosity models')
plt.suptitle('Comparison of glacial Young Fraction\n due to heterogeneous porosity, by {}'.format(category), fontsize=14)
#ax.text(0.76, maxi, 'R2 = {:3.3f}'.format(r2_y1), fontsize=9, color='green')
#ax.text(0.76, maxi-0.02, 'R2 = {:3.3f}'.format(r2_y2), fontsize=9, color='blue')
ax.text(lowlim+0.03*lrange, uplim-0.06*lrange, r'$r_p$ = {:3.3f}'.format(Prho_y1), fontsize=9, color='green')
ax.text(lowlim+0.03*lrange, uplim-0.11*lrange, r'$r_s$ = {:3.3f}'.format(Srho_y1), fontsize=9, color='green')
ax.text(lowlim+0.03*lrange, uplim-0.16*lrange, r'$std$ = {:3.3f}'.format(std_y1), fontsize=9, color='green')
ax.text(lowlim+0.03*lrange, uplim-0.21*lrange, r'$ME$ = {:3.3f}'.format(ME_y1), fontsize=9, color='green')
ax.text(lowlim+0.03*lrange, uplim-0.26*lrange, r'$r_p$ = {:3.3f}'.format(Prho_y2), fontsize=9, color='blue')
ax.text(lowlim+0.03*lrange, uplim-0.31*lrange, r'$r_s$ = {:3.3f}'.format(Srho_y2), fontsize=9, color='blue')
ax.text(lowlim+0.03*lrange, uplim-0.36*lrange, r'$std$ = {:3.3f}'.format(std_y2), fontsize=9, color='blue')
ax.text(lowlim+0.03*lrange, uplim-0.41*lrange, r'$ME$ = {:3.3f}'.format(ME_y2), fontsize=9, color='blue')
#plt.tight_layout()

dst = 'YFrac_121_compare_heteroN_{}'.format(category)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of Young Fraction for uniform (X) versus heterogeneous (Y) porosity for ALL HUC scales.

x1, y1, x2, y2 = eptlist[2], eptlist[3], eptlist[0], eptlist[1]  # columns in eptlist. In this case (x=uniform n; y = hetero n)
hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)
fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axisdict = {0:ax1, 1:ax2, 2:ax3, 3:ax4}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[2]][each].unique()
    u = np.append(u1, u2)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    axis = axisdict[k]

    yfhucdict = {}
    yfvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques):  # each HUC ID
        yfmoddict = {}
        for i, md in enumerate(eptlist):  # each of the 3 FWP models
            mn = eptlist[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                yf = youngdf.rt.count() / df.rt.count()
                yfmoddict[mn] = yf
                yfvalues.append(yf)
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict    

    ddd = pd.DataFrame(yfhucdict).T
    Prho_y1, Pp_y1 = stats.pearsonr(np.array(ddd[x1]), np.array(ddd[y1]))
    Srho_y1, Sp_y1 = stats.spearmanr(np.array(ddd[x1]), np.array(ddd[y1]))
    std_y1 = stats.tstd(np.array(ddd[x1]) - np.array(ddd[y1]))
    ME_y1 = np.mean(np.array(ddd[x1]) - np.array(ddd[y1]))
    Prho_y2, Pp_y2 = stats.pearsonr(np.array(ddd[x2]), np.array(ddd[y2]))
    Srho_y2, Sp_y2 = stats.spearmanr(np.array(ddd[x2]), np.array(ddd[y2]))
    std_y2 = stats.tstd(np.array(ddd[x2]) - np.array(ddd[y2]))
    ME_y2 = np.mean(np.array(ddd[x2]) - np.array(ddd[y2]))
    
    #plotting
    mini = min(min(ddd[x1]), min(ddd[x2]), min(ddd[y1]), min(ddd[y2]))
    maxi = max(max(ddd[x1]), max(ddd[x2]), max(ddd[y1]), max(ddd[y2]))
    yrange = maxi-mini
    if k == 0:
        ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='{}: uniform n vs heterogeneous n'.format(evaluate[1]), ax=axis)
        ddd.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', label = '{}: uniform n vs heterogeneous n'.format(evaluate[0]), ax=axis)
        axis.text(mini, mini+0.83*yrange, r'$r_p$ = {:3.3f}'.format(Prho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.78*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.73*yrange, r'$std$ = {:3.3f}'.format(std_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.68*yrange, r'$ME$ = {:3.3f}'.format(ME_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.63*yrange, r'$r_p$ = {:3.3f}'.format(Prho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.58*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.53*yrange, r'$std$ = {:3.3f}'.format(std_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.48*yrange, r'$ME$ = {:3.3f}'.format(ME_y2), fontsize=9, color='blue')
    else:
        ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=axis)
        ddd.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', ax=axis)
        axis.text(mini, mini+yrange, r'$r_p$ = {:3.3f}'.format(Prho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.95*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.90*yrange, r'$std$ = {:3.3f}'.format(std_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.85*yrange, r'$ME$ = {:3.3f}'.format(ME_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.80*yrange, r'$r_p$ = {:3.3f}'.format(Prho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.75*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.70*yrange, r'$std$ = {:3.3f}'.format(std_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.65*yrange, r'$ME$ = {:3.3f}'.format(ME_y2), fontsize=9, color='blue')    
    
    axis.plot((mini, maxi), (mini, maxi), 'k--')
    axis.set_xlabel(''), axis.set_ylabel('')
    axis.set_title(each)
    
    # Add an attribute to the HUC shapefile
    shapefile = HUCshpdict[each]
    shp = gpd.read_file(shapefile)  
    shp[each] = shp[each].astype('int64')  # convert so can merge
    ddd.index = ddd.index.astype('int64')  # convert so can merge
    ddd['YF_zK_uniVShet_n'.format(scenario_name, base_case)] = ddd[x1] - ddd[y1]
    ddd['YF_hK_uniVShet_n'.format(scenario_name, base_case)] = ddd[x2] - ddd[y2]

    df = pd.merge(shp, ddd, how='outer', left_on=each, right_index=True)
    dst = shapefile[:-4] + '_YF_{}.shp'.format(scenario_name)
    df.to_file(dst)

fig.text(0.5, 0.06, 'Young Fraction from the uniform porosity models', ha='center')
fig.text(0.06, 0.48, 'Young Fraction from the heterogeneous porosity models', va='center', rotation='vertical')
plt.subplots_adjust(top = .9)
fig.suptitle('Comparison of glacial aquifer Young Fractions\n by HUCs: uniform vs heterogeneous porosity.', fontsize=14)

dst = 'YFrac_121_uniVShet_n_allHUCS'
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
eptlist

In [ ]:
# 1:1 plot of Young Fraction for complexe (X) versus simple (Y)  MF models for uniform and heterogeneous porosities for all HUC scales

# That is, we're testing to see if adding heterogeneous porosity increases or decreases the mis-match between simple and complexe MF models.
# If rho, std, & ME all improve or degrade, then that may indicate if age metrics from simple MF models can be improved by using texture to inform porosity.

x1, y1, x2, y2 = eptlist[0], eptlist[2], eptlist[1], eptlist[3]  # columns in eptlist. In this case (x=complexe MF; y=simple MF)
hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)
fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axisdict = {0:ax1, 1:ax2, 2:ax3, 3:ax4}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[2]][each].unique()
    u = np.append(u1, u2)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    axis = axisdict[k]

    yfhucdict = {}
    yfvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques):  # each HUC ID
        yfmoddict = {}
        for i, md in enumerate(eptlist):  # each of the 3 FWP models
            mn = eptlist[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                yf = youngdf.rt.count() / df.rt.count()
                yfmoddict[mn] = yf
                yfvalues.append(yf)
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict    

    ddd = pd.DataFrame(yfhucdict).T
    Prho_y1, Pp_y1 = stats.pearsonr(np.array(ddd[x1]), np.array(ddd[y1]))
    Srho_y1, Sp_y1 = stats.spearmanr(np.array(ddd[x1]), np.array(ddd[y1]))
    std_y1 = stats.tstd(np.array(ddd[x1]) - np.array(ddd[y1]))
    ME_y1 = np.mean(np.array(ddd[x1]) - np.array(ddd[y1]))
    Prho_y2, Pp_y2 = stats.pearsonr(np.array(ddd[x2]), np.array(ddd[y2]))
    Srho_y2, Sp_y2 = stats.spearmanr(np.array(ddd[x2]), np.array(ddd[y2]))
    std_y2 = stats.tstd(np.array(ddd[x2]) - np.array(ddd[y2]))
    ME_y2 = np.mean(np.array(ddd[x2]) - np.array(ddd[y2]))
    
    #plotting
    mini = min(min(ddd[x1]), min(ddd[x2]), min(ddd[y1]), min(ddd[y2]))
    maxi = max(max(ddd[x1]), max(ddd[x2]), max(ddd[y1]), max(ddd[y2]))
    yrange = maxi-mini
    if k == 0:
        ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='uniform n: hetero. K vs zoned K', ax=axis)
        ddd.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', label = 'hetero. n: hetero. K vs zoned K', ax=axis)
        axis.text(mini, mini+0.83*yrange, r'$r_p$ = {:3.3f}'.format(Prho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.78*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.73*yrange, r'$std$ = {:3.3f}'.format(std_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.68*yrange, r'$ME$ = {:3.3f}'.format(ME_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.63*yrange, r'$r_p$ = {:3.3f}'.format(Prho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.58*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.53*yrange, r'$std$ = {:3.3f}'.format(std_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.48*yrange, r'$ME$ = {:3.3f}'.format(ME_y2), fontsize=9, color='blue')
    else:
        ddd.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=axis)
        ddd.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', ax=axis)
        axis.text(mini, mini+yrange, r'$r_p$ = {:3.3f}'.format(Prho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.95*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.90*yrange, r'$std$ = {:3.3f}'.format(std_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.85*yrange, r'$ME$ = {:3.3f}'.format(ME_y1), fontsize=9, color='green')
        axis.text(mini, mini+0.80*yrange, r'$r_p$ = {:3.3f}'.format(Prho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.75*yrange, r'$r_s$ = {:3.3f}'.format(Srho_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.70*yrange, r'$std$ = {:3.3f}'.format(std_y2), fontsize=9, color='blue')
        axis.text(mini, mini+0.65*yrange, r'$ME$ = {:3.3f}'.format(ME_y2), fontsize=9, color='blue')    
    
    axis.plot((mini, maxi), (mini, maxi), 'k--')
    axis.set_xlabel(''), axis.set_ylabel('')
    axis.set_title(each)
    
    # Add an attribute to the HUC shapefile
    shapefile = HUCshpdict[each]
    shp = gpd.read_file(shapefile)  
    shp[each] = shp[each].astype('int64')  # convert so can merge
    ddd.index = ddd.index.astype('int64')  # convert so can merge
    #ddd['YF_zK_uniVShet_n'.format(scenario_name, base_case)] = ddd[x1] - ddd[y1]
    ddd['YF_{}_hK2zK'.format(scenario_name)] = ddd[x2] - ddd[y2]

    df = pd.merge(shp, ddd, how='outer', left_on=each, right_index=True)
    dst = shapefile[:-4] + '_YF_{}_hK2zK.shp'.format(scenario_name)
    df.to_file(dst)

fig.text(0.5, 0.06, 'Young Fraction from the heterogeneous K Modflow models', ha='center')
fig.text(0.06, 0.48, 'Young Fraction from the zoned K Modflow models', va='center', rotation='vertical')
plt.subplots_adjust(top = .9)
fig.suptitle('Comparison of glacial aquifer Young Fractions\n by HUCs: heterogeneous vs zoned K.', fontsize=14)

dst = 'YFrac_121_hetn_hK2zK_allHUCS'
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of median-YF-age and mOFage for complexe (X) versus simple (Y) MF models for uniform and heterogeneous porosities 
# for the select scale.

# That is, does adding heterogeneous porosity also improve median ages as it did with YF calculations for simple and complex
# MF models?

x1, y1, x2, y2 = eptlist[0], eptlist[2], eptlist[1], eptlist[3]  # columns in eptlist. In this case (x=complexe MF; y=simple MF)

# make sure we get ALL unique HUC IDs
u1 = dfdict[eptlist[0]][category].unique()
u2 = dfdict[eptlist[1]][category].unique()
u = np.append(u1, u2)
uniques = np.unique(u)
# remove any HUCs listed in purge_hucs
for h in purge_hucs:
    ind = np.where(uniques==h)
    uniques = np.delete(uniques, ind)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4.6))

skiphuc = []
yfhucdict = {}
yfvalues = []
ofhucdict = {}
ofvalues = []
for j, cat_val in enumerate(uniques):  # each HUC ID
    yfmoddict = {}
    ofmoddict = {}
    for i, md in enumerate(eptlist):  # each of the 3 FWP models
        mn = eptlist[i]
        df = dfdict[md].loc[dfdict[md][category]==cat_val]
        if df.rt.count() >= minptl:
            youngdf = df.loc[df.rt < age_cutoff]
            olddf = df.loc[df.rt >= age_cutoff]
            yfage = youngdf.rt.median()
            ofage = olddf.rt.median()
            yfmoddict[mn] = yfage
            yfvalues.append(yfage) 
            ofmoddict[mn] = ofage
            ofvalues.append(ofage)
        else:
            skiphuc.append(cat_val)
            break
    if cat_val not in skiphuc:
        yfhucdict[cat_val] = yfmoddict
        ofhucdict[cat_val] = ofmoddict
        
young = pd.DataFrame(yfhucdict).T
old = pd.DataFrame(ofhucdict).T
young.dropna(axis='index', how='any', inplace=True)
old.dropna(axis='index', how='any', inplace=True)
    
Prho_y1y, Pp_y1y = stats.pearsonr(np.array(young[x1]), np.array(young[y1]))
Srho_y1y, Sp_y1y = stats.spearmanr(np.array(young[x1]), np.array(young[y1]))
std_y1y = stats.tstd(np.array(young[x1]) - np.array(young[y1]))
ME_y1y = np.mean(np.array(young[x1]) - np.array(young[y1]))
Prho_y2y, Pp_y2y = stats.pearsonr(np.array(young[x2]), np.array(young[y2]))
Srho_y2y, Sp_y2y = stats.spearmanr(np.array(young[x2]), np.array(young[y2]))
std_y2y = stats.tstd(np.array(young[x2]) - np.array(young[y2]))
ME_y2y = np.mean(np.array(young[x2]) - np.array(young[y2]))
Prho_y1o, Pp_y1o = stats.pearsonr(np.array(old[x1]), np.array(old[y1]))
Srho_y1o, Sp_y1o = stats.spearmanr(np.array(old[x1]), np.array(old[y1]))
std_y1o = stats.tstd(np.array(old[x1]) - np.array(old[y1]))
ME_y1o = np.mean(np.array(old[x1]) - np.array(old[y1]))
Prho_y2o, Pp_y2o = stats.pearsonr(np.array(old[x1]), np.array(old[y2]))
Srho_y2o, Sp_y2o = stats.spearmanr(np.array(old[x1]), np.array(old[y2]))
std_y2o = stats.tstd(np.array(old[x2]) - np.array(old[y2]))
ME_y2o = np.mean(np.array(old[x2]) - np.array(old[y2]))    

#plotting
miny, maxy = min(yfvalues), max(yfvalues)
mino, maxo = min(ofvalues), max(ofvalues)
yrange_y = maxy-miny
yrange_o = maxo-mino

young.plot(kind='scatter', x=x1, y=y1, marker='o', c='green', label='uniform n: hetero. K vs zoned K', ax=ax1)
young.plot(kind='scatter', x=x2, y=y2, marker='^', c='blue', label='hetero. n: hetero. K vs zoned K', ax=ax1)
old.plot(kind='scatter', x=x1, y=y1, marker='o', c='green', ax=ax2)
old.plot(kind='scatter', x=x2, y=y2, marker='^', c='blue', ax=ax2)
ax1.text(miny, miny+0.80*yrange_y, r'$r_p$ = {:3.3f}'.format(Prho_y1y), fontsize=9, color='green')
ax1.text(miny, miny+0.75*yrange_y, r'$r_s$ = {:3.3f}'.format(Srho_y1y), fontsize=9, color='green')
ax1.text(miny, miny+0.70*yrange_y, r'$std$ = {:3.3f}'.format(std_y1y), fontsize=9, color='green')
ax1.text(miny, miny+0.65*yrange_y, r'$ME$ = {:3.3f}'.format(ME_y1y), fontsize=9, color='green')
ax1.text(miny, miny+0.60*yrange_y, r'$r_p$ = {:3.3f}'.format(Prho_y2y), fontsize=9, color='blue')
ax1.text(miny, miny+0.55*yrange_y, r'$r_s$ = {:3.3f}'.format(Srho_y2y), fontsize=9, color='blue')
ax1.text(miny, miny+0.50*yrange_y, r'$std$ = {:3.3f}'.format(std_y2y), fontsize=9, color='blue')
ax1.text(miny, miny+0.45*yrange_y, r'$ME$ = {:3.3f}'.format(ME_y2y), fontsize=9, color='blue')
ax2.text(mino, mino+yrange_o, r'$r_p$ = {:3.3f}'.format(Prho_y1o), fontsize=9, color='green')
ax2.text(mino, mino+0.95*yrange_o, r'$r_s$ = {:3.3f}'.format(Srho_y1o), fontsize=9, color='green')
ax2.text(mino, mino+0.90*yrange_o, r'$std$ = {:3.3f}'.format(std_y1o), fontsize=9, color='green')
ax2.text(mino, mino+0.85*yrange_o, r'$ME$ = {:3.3f}'.format(ME_y1o), fontsize=9, color='green')
ax2.text(mino, mino+0.80*yrange_o, r'$r_p$ = {:3.3f}'.format(Prho_y2o), fontsize=9, color='blue')
ax2.text(mino, mino+0.75*yrange_o, r'$r_s$ = {:3.3f}'.format(Srho_y2o), fontsize=9, color='blue')
ax2.text(mino, mino+0.70*yrange_o, r'$std$ = {:3.3f}'.format(std_y2o), fontsize=9, color='blue')
ax2.text(mino, mino+0.65*yrange_o, r'$ME$ = {:3.3f}'.format(ME_y2o), fontsize=9, color='blue')

ax1.plot((miny, maxy), (miny, maxy), 'k--')
ax2.plot((mino, maxo), (mino, maxo), 'k--')
ax1.set_xlabel(''), ax1.set_ylabel(''), ax2.set_ylabel(''), ax2.set_xlabel('')
fig.text(0.25, 0.82, 'Young Water')
fig.text(0.73, 0.82, 'Old Water', ha='center')
fig.text(0.07, 0.45, 'Median age (yrs) for the zoned K Modflow models', va='center', rotation='vertical')
fig.text(0.5, 0.01, 'Median age (yrs) for the heterogeneous K Modflow models', ha='center')
plt.subplots_adjust(top = .8)
fig.suptitle('Comparison of median ages for young and\n old water in the glacial aquifer by {}: heterogeneous vs zoned K.'.format(category), fontsize=14)
#fig.tight_layout()

dst = 'YnOages_121_hetn_hK2zK_{}'.format(category)
dst_pth = os.path.join(fig_dir, dst)
plt.savefig(dst_pth)
#plt.close()

In [ ]:
# 1:1 plot of median-YF-age and mOFage for complex (X) versus simple (Y) MF models for uniform and heterogeneous porosities 
# for ALL HUC scales

# That is, does adding heterogeneous porosity also improve median ages as it did with YF calculations for simple and complex
# MF models?

x1, y1, x2, y2 = eptlist[0], eptlist[2], eptlist[1], eptlist[3]  # columns in eptlist. In this case (x=complexe MF; y=simple MF)

hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)
fig1, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
fig2, [[ax5, ax6], [ax7, ax8]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
youngaxisdict = {0:ax1, 1:ax2, 2:ax3, 3:ax4}
oldaxisdict = {0:ax5, 1:ax6, 2:ax7, 3:ax8}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[1]][each].unique()
    u = np.append(u1, u2)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    youngaxis = youngaxisdict[k]
    oldaxis = oldaxisdict[k]
    ofhucdict = {}
    yfhucdict = {}
    yfvalues = []
    ofvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques):  # each HUC ID
        yfmoddict = {} 
        ofmoddict = {} 
        for i, md in enumerate(eptlist):  # each of the 3 FWP models
            mn = eptlist[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                olddf = df.loc[df.rt >= age_cutoff]
                yfage = youngdf.rt.median()
                ofage = olddf.rt.median()
                yfmoddict[mn] = yfage
                yfvalues.append(yfage) 
                ofmoddict[mn] = ofage
                ofvalues.append(ofage)                
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict
            ofhucdict[cat_val] = ofmoddict            
            
    young = pd.DataFrame(yfhucdict).T
    old = pd.DataFrame(ofhucdict).T
    young.dropna(axis='index', how='any', inplace=True)  # better way to remove NANs
    old.dropna(axis='index', how='any', inplace=True)
    Prho_y1y, Pp_y1y = stats.pearsonr(np.array(young[x1]), np.array(young[y1]))
    Srho_y1y, Sp_y1y = stats.spearmanr(np.array(young[x1]), np.array(young[y1]))
    std_y1y = stats.tstd(np.array(young[x1]) - np.array(young[y1]))
    ME_y1y = np.mean(np.array(young[x1]) - np.array(young[y1]))
    Prho_y2y, Pp_y2y = stats.pearsonr(np.array(young[x2]), np.array(young[y2]))
    Srho_y2y, Sp_y2y = stats.spearmanr(np.array(young[x2]), np.array(young[y2]))
    std_y2y = stats.tstd(np.array(young[x2]) - np.array(young[y2]))
    ME_y2y = np.mean(np.array(young[x2]) - np.array(young[y2]))
    Prho_y1o, Pp_y1o = stats.pearsonr(np.array(old[x1]), np.array(old[y1]))
    Srho_y1o, Sp_y1o = stats.spearmanr(np.array(old[x1]), np.array(old[y1]))
    std_y1o = stats.tstd(np.array(old[x1]) - np.array(old[y1]))
    ME_y1o = np.mean(np.array(old[x1]) - np.array(old[y1]))
    Prho_y2o, Pp_y2o = stats.pearsonr(np.array(old[x1]), np.array(old[y2]))
    Srho_y2o, Sp_y2o = stats.spearmanr(np.array(old[x1]), np.array(old[y2]))
    std_y2o = stats.tstd(np.array(old[x2]) - np.array(old[y2]))
    ME_y2o = np.mean(np.array(old[x2]) - np.array(old[y2]))     
    
    #plotting
    miny, maxy = min(yfvalues), max(yfvalues)
    mino, maxo = min(ofvalues), max(ofvalues)
    yrange_y = maxy-miny
    yrange_o = maxo-mino
    
    if k == 0:
        young.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='uniform n: hetero. K vs zoned K', ax=youngaxis)
        young.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', label = 'hetero. n: hetero. K vs zoned K', ax=youngaxis)
        
        old.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', label='uniform n: hetero. K vs zoned K', ax=oldaxis)
        old.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', label = 'hetero. n: hetero. K vs zoned K', ax=oldaxis)  
        #youngaxis.text(miny, miny+0.83*yrange_y, 'R2 = {:3.3f}'.format(r2_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.83*yrange_y, r'$r_p$ = {:3.3f}'.format(Prho_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.78*yrange_y, r'$r_s$ = {:3.3f}'.format(Srho_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.73*yrange_y, r'$std$ = {:3.3f}'.format(std_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.68*yrange_y, r'$ME$ = {:3.3f}'.format(ME_y1y), fontsize=9, color='green')
        #youngaxis.text(miny, miny+0.78*yrange_y, 'R2 = {:3.3f}'.format(r2_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.63*yrange_y, r'$r_p$ = {:3.3f}'.format(Prho_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.58*yrange_y, r'$r_s$ = {:3.3f}'.format(Srho_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.53*yrange_y, r'$std$ = {:3.3f}'.format(std_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.48*yrange_y, r'$ME$ = {:3.3f}'.format(ME_y2y), fontsize=9, color='blue')
        #oldaxis.text(mino, mino+0.83*yrange_o, 'R2 = {:3.3f}'.format(r2_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.83*yrange_o, r'$r_p$ = {:3.3f}'.format(Prho_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.78*yrange_o, r'$r_s$ = {:3.3f}'.format(Srho_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.73*yrange_o, r'$std$ = {:3.3f}'.format(std_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.68*yrange_o, r'$ME$ = {:3.3f}'.format(ME_y1o), fontsize=9, color='green')
        #oldaxis.text(mino, mino+0.78*yrange_o, 'R2 = {:3.3f}'.format(r2_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.63*yrange_o, r'$r_p$ = {:3.3f}'.format(Prho_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.58*yrange_o, r'$r_s$ = {:3.3f}'.format(Srho_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.53*yrange_o, r'$std$ = {:3.3f}'.format(std_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.48*yrange_o, r'$ME$ = {:3.3f}'.format(ME_y2o), fontsize=9, color='blue')
    else:
        young.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=youngaxis)
        young.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', ax=youngaxis)
        old.plot(kind='scatter', x=x1, y=y1, marker='o', color='green', ax=oldaxis)
        old.plot(kind='scatter', x=x2, y=y2, marker='^', color='blue', ax=oldaxis)     
        youngaxis.text(miny, miny+yrange_y, r'$r_p$ = {:3.3f}'.format(Prho_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.95*yrange_y, r'$r_s$ = {:3.3f}'.format(Srho_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.90*yrange_y, r'$std$ = {:3.3f}'.format(std_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.85*yrange_y, r'$ME$ = {:3.3f}'.format(ME_y1y), fontsize=9, color='green')
        youngaxis.text(miny, miny+0.80*yrange_y, r'$r_p$ = {:3.3f}'.format(Prho_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.75*yrange_y, r'$r_s$ = {:3.3f}'.format(Srho_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.70*yrange_y, r'$std$ = {:3.3f}'.format(std_y2y), fontsize=9, color='blue')
        youngaxis.text(miny, miny+0.65*yrange_y, r'$ME$ = {:3.3f}'.format(ME_y2y), fontsize=9, color='blue')
        oldaxis.text(mino, mino+yrange_o, r'$r_p$ = {:3.3f}'.format(Prho_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.95*yrange_o, r'$r_s$ = {:3.3f}'.format(Srho_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.90*yrange_o, r'$std$ = {:3.3f}'.format(std_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.85*yrange_o, r'$ME$ = {:3.3f}'.format(ME_y1o), fontsize=9, color='green')
        oldaxis.text(mino, mino+0.80*yrange_o, r'$r_p$ = {:3.3f}'.format(Prho_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.75*yrange_o, r'$r_s$ = {:3.3f}'.format(Srho_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.70*yrange_o, r'$std$ = {:3.3f}'.format(std_y2o), fontsize=9, color='blue')
        oldaxis.text(mino, mino+0.65*yrange_o, r'$ME$ = {:3.3f}'.format(ME_y2o), fontsize=9, color='blue')
        
    youngaxis.plot((miny, maxy), (miny, maxy), 'k--')
    oldaxis.plot((mino, maxo), (mino, maxo), 'k--')

    youngaxis.set_xlabel(''), youngaxis.set_ylabel(''), oldaxis.set_xlabel(''), oldaxis.set_ylabel('')
    youngaxis.set_title(each)
    oldaxis.set_title(each)
    
    # Add an attribute to the HUC shapefile
    #shapefile = HUCshpdict[each]
    #shp = gpd.read_file(shapefile)  
    #shp[each] = shp[each].astype('int64')  # convert so can merge
    
    #young.index = young.index.astype('int64')  # convert so can merge
    #young['mYage_{}-{}'.format(scenario_name, base_case)] = young[x1] - young[y1]
    ##young['mYage_hK-zK'] = young[x1] - young[y2]
    #df = pd.merge(shp, young, how='outer', left_on=each, right_index=True)
    #dst = shapefile[:-4] + '_mYage_{}.shp'.format(scenario_name)
    #df.to_file(dst)
    
    #old.index = old.index.astype('int64')  # convert so can merge
    #old['mOage_{}-{}'.format(scenario_name, base_case)] = old[x1] - old[y1]
    ##old['mOage_hK-zK'] = old[x1] - old[y2]
    #dfo = pd.merge(shp, old, how='outer', left_on=each, right_index=True)
    #dsto = shapefile[:-4] + '_mOage_{}.shp'.format(scenario_name)
    #dfo.to_file(dsto)

#fig1.text(0.46, 0.92, 'Young fraction', fontsize=12)
#fig.text(0.71, 0.92, 'Old fraction', ha='center', fontsize=12)
#fig2.text(0.48, 0.92, 'Old fraction', fontsize=12)
fig1.text(0.5, 0.055, 'Median age (yrs) for the 5-layer heterogeneous K Modflow models', ha='center', fontsize=12)
fig1.text(0.001, 0.48, 'Median age (yrs) for the 5-layer zoned K Modflow models', va='center', fontsize=12, rotation='vertical')
fig2.text(0.5, 0.055, 'Median age (yrs) for the 5-layer heterogeneous K Modflow models', ha='center', fontsize=12)
fig2.text(0.005, 0.48, 'Median age (yrs) for the 5-layer zoned K Modflow models', va='center', fontsize=12, rotation='vertical')
plt.subplots_adjust(top = .89)
fig1.suptitle('Comparison of median ages for young fraction in the\n glacial aquifer by HUCs: heterogeneous vs zoned K.', fontsize=16)
fig2.suptitle('Comparison of median ages for old fraction in the\n glacial aquifer by HUCs: heterogeneous vs zoned K.', fontsize=16)

dst1 = 'Yages_121_hetn_hK2zK_allHUCs'
dst2 = 'Oages_121_hetn_hK2zK_allHUCs'
dst_pth1 = os.path.join(fig_dir, dst1)
dst_pth2 = os.path.join(fig_dir, dst2)
fig1.savefig(dst_pth1)
fig2.savefig(dst_pth2)
#plt.close()

In [ ]:
from matplotlib.colors import Normalize

#  https://stackoverflow.com/questions/20144529/shifted-colorbar-matplotlib/20146989#20146989
class MidpointNormalize(Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

In [ ]:
# populate DF with zonal stats
def hucpropstats2(hucrast, proprast, dataframe, raster_file, metric, descriptor):  # fluxtype, mod_factor
    #descriptor = '{}_{}_{}'.format(metric, fluxtype, mod_factor)
    df = dataframe.copy()
    #df[metric] = 0  # initialize
    df[descriptor] = 0  # initialize
    u = np.unique(hucrast)
    if max(u) < 10000:  # Less than any HUC6 ID number
        with open(raster_file.split('.')[0] + '_crossref.txt', mode='r') as infile:
            reader = csv.reader(infile)
            next(reader, None)  # skip the headers
            crossdict = {int(rows[0]):np.int64(rows[1]) for rows in reader}

    for ID in u[u>0]:  # HUC IDs in active part of model (not ID zero)
        idarr = np.zeros_like(hucrast)
        idarr[hucrast==ID] = 1
        hucprop = idarr * proprast
        hucprop[hucprop==0] = np.nan
        pmn, pmed, psum, pmin, pmax, pstd, pcount = np.nanmean(hucprop), np.nanmedian(hucprop), np.nansum(hucprop), np.nanmin(hucprop), np.nanmax(hucprop), np.nanstd(hucprop), np.count_nonzero(~np.isnan(hucprop))
        prange = pmax - pmin
        metricdict = {'mean':pmn, 'median':pmed, 'sum':psum, 'min':pmin, 'max':pmax, 'range':prange, 'std':pstd, 'count':pcount}
        if max(u) < 10000:
            # assigned the appropriate zonal data
            #df[metric].loc[df.index==crossdict[ID]] = metricdict[metric].astype('float64')              
            try:
                df[descriptor].loc[df.index==crossdict[ID]] = metricdict[metric].astype('float64')              
            except:
                df[descriptor].loc[df.index==crossdict[ID]] = int(metricdict[metric])
        else:
            # assigned the appropriate zonal data
            #df[metric].loc[df.index==ID] = metricdict[metric].astype('float64')  
            try:
                df[descriptor].loc[df.index==ID] = metricdict[metric].astype('float64')  
            except:
                df[descriptor].loc[df.index==ID] = int(metricdict[metric])
    return(df)

In [ ]:
# Evaluate model factors (weak sinks) when using heterogeneous n.  Does the pattern seen with uniform n still hold?

# reduce the analysis to just the heterogeneous n results
hetn_list = []
hetn_list.append(eptlist[1])
hetn_list.append(eptlist[3])
x1, y1 = hetn_list[0], hetn_list[1]  # columns in eptlist. In this case (x=complexe MF; y=simple MF)

# 1:1 plot of YoungFraction btwn 5LhK and 5LzK models for all HUC scales; color coded by differences in model output
# among the 5LhK and 5LzK models.  What's causing the YF differences?

topic = 'discharge'  # Chose between 'discharge' or 'seepage'.  Refers to dicts of tiffs for discharge to SFR cells or SFR cells seeping into aquifers
metric = 'count'  # valid:  'mean', 'median', sum', min', 'max', 'range', 'std', 'count'

hucs = ['HUC6', 'HUC8', 'HUC10', 'HUC12']
nhucs = len(hucs)

# loop this thing
#for prop in HUCfluxrastlist:

#for Qkey, prop in dischargerastdict.items():
fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axisdict = {0:ax1, 1:ax2, 2:ax3, 3:ax4}

for k, each in enumerate(hucs):
    # make sure we get ALL unique HUC IDs
    u1 = dfdict[eptlist[0]][each].unique()
    u2 = dfdict[eptlist[1]][each].unique()
    u3 = dfdict[eptlist[2]][each].unique()
    u = np.append(u1, u2)
    u = np.append(u, u3)
    uniques = np.unique(u)
    # remove any HUCs listed in purge_hucs
    for h in purge_hucs:
        ind = np.where(uniques==h)
        uniques = np.delete(uniques, ind)
    axis = axisdict[k]

    yfhucdict = {}
    yfvalues = []
    skiphuc = []
    for j, cat_val in enumerate(uniques.astype('int64')):  # each HUC ID
        yfmoddict = {}            
        #for i, md in enumerate(eptlist):  # each of the 3 FWP models
        for i, md in enumerate(hetn_list):  # each of the 3 FWP models
            mn = hetn_list[i]
            df = dfdict[md].loc[dfdict[md][each]==cat_val]
            if df.rt.count() >= minptl:
                youngdf = df.loc[df.rt < age_cutoff]
                yf = youngdf.rt.count() / df.rt.count()
                yfmoddict[mn] = yf
                yfvalues.append(yf)
            else:
                skiphuc.append(cat_val)
                break
        if cat_val not in skiphuc:
            yfhucdict[cat_val] = yfmoddict    
    ddd = pd.DataFrame(yfhucdict).T
    for n in hetn_list:
        ddd = ddd.loc[~ddd[n].isnull()]  # need to remove any NANs
        
    # run zonal_stats on the merged df. Use the 'HUC' Tiffs to match up with model properties
    print('Processing HUCs for {}'.format(each))
    raster_file = HUCtiffdict[each]
    with rasterio.open(raster_file) as raster:
        hucrast = raster.read()[0]
        
    #Process the "evaluate flux rasters"
    for i in range(len(evaluate)):
        modnm = evaluate[i]
        if topic == 'discharge':
            prop = dischargerastdict[modnm]
        elif topic == 'seepage':
            prop = lossrastdict[modnm]
        else:
            print('only "discharge" or "seepage" are valid entries for the "topic" variable.  Stopping.')
            sys.exit()
        fluxtype = os.path.basename(prop).split('.')[0].split('_')[-1]  # type of info in the rasters
        print('Extracting zonal summary values from {} for {}'.format(prop, each))
        with rasterio.open(prop) as raster:
            fluxrast = raster.read()[0]
            if np.nanmax(fluxrast) <= 0.0:  # convert all negative arrays to positive
                fluxrast = fluxrast * -1
        if i == 0:
            mod_factor1 = abrevdict[modnm]
            descript1 = '{}_{}_{}'.format(metric, fluxtype, mod_factor1)
            firstflux = fluxrast.copy() 
            ddd1 = hucpropstats2(hucrast, firstflux, ddd, raster_file, metric, descript1) 
        elif i==1:
            mod_factor2 = abrevdict[modnm]
            descript2 = '{}_{}_{}'.format(metric, fluxtype, mod_factor2)
            # process second raster.
            ddd2 = hucpropstats2(hucrast, fluxrast, ddd1, raster_file, metric, descript2)
            fluxdiff = firstflux - fluxrast
            mod_factor3 = mod_factor1 + '-' + mod_factor2
            descript3 = '{}_{}_{}'.format(metric, fluxtype, mod_factor3)
            ddd2[descript3] = ddd2[descript1] - ddd2[descript2]
            descript4 = '%change_{}'.format(descript3)
            ddd2[descript4] = (ddd2[descript3] / ddd2[descript1]) * 100.0
            ddd2.replace([np.inf, -np.inf], np.nan)  # just in case infinity is calculated. Convert to nan before purge nan.
            ddd2 = ddd2.loc[ddd2[descript4].notnull()]  # remove any NANs
        else:
            print('Only 2 models were supposed to be listed in the EVALUATE variable.  Please fix that and re-run. Stopping.')
            sys.exit()
    
    print('Preparing plots for {}'.format(each))
    ddd2[mod_factor3] = ddd2[x1] - ddd2[y1]  # AGE difference
    Srho_color, Sp_color = stats.spearmanr(np.array(ddd2[descript4]), np.array(ddd2[mod_factor3]))
    Prho_color, Pp_color = stats.pearsonr(np.array(ddd2[descript4]), np.array(ddd2[mod_factor3]))
    std_color = stats.tstd(np.array(ddd2[descript4]) - np.array(ddd2[mod_factor3]))
    ME_color = np.mean(np.array(ddd2[descript4]) - np.array(ddd2[mod_factor3]))
         
    #plotting
    mini = min(min(ddd2[x1]), min(ddd2[y1]))
    maxi = max(max(ddd2[x1]), max(ddd2[y1]))
    yrange = maxi-mini
    norm = MidpointNormalize(midpoint=0)
    ddd2.plot(kind='scatter', x=x1, y=y1, marker='^', c=ddd2[descript4], cmap='RdYlBu', norm=norm, ax=axis, vmax=ddd2[descript4].quantile(.99), vmin=ddd2[descript4].quantile(.01))
    lowlim = min(axis.get_xlim()[0], axis.get_ylim()[0])
    uplim = max(axis.get_xlim()[1], axis.get_ylim()[1])
    lims = (lowlim, uplim)
    lrange = uplim - lowlim
    axis.set_xlim(lims)  # force the plots to be square
    axis.set_ylim(lims)
    axis.text(lowlim+0.03*lrange, uplim-0.11*lrange, 'Fit metrics between model property \nand YF differences:', fontsize=9)
    axis.text(lowlim+0.03*lrange, uplim-0.16*lrange, r'$r_p$ = {:3.3f}  $p$={:1.1e}'.format(Prho_color, Pp_color), fontsize=9)
    axis.text(lowlim+0.03*lrange, uplim-0.21*lrange, r'$r_s$ = {:3.3f}  $p$={:1.1e}'.format(Srho_color, Sp_color), fontsize=9)
    axis.text(lowlim+0.03*lrange, uplim-0.26*lrange, r'$std$ = {:3.3f}'.format(std_color), fontsize=9)
    axis.text(lowlim+0.03*lrange, uplim-0.31*lrange, r'$ME$ = {:3.3f}'.format(ME_color), fontsize=9)
    pos1 = axis.get_position()
    axis.plot((mini, maxi), (mini, maxi), 'k--')
    axis.annotate("", xy=(mini+0.32*yrange, maxi-0.35*yrange), xytext=(mini+0.48*yrange, mini+0.52*yrange), arrowprops=dict(arrowstyle="->", lw=3, color='gray', alpha=0.3))  # orange
    axis.annotate("", xy=(maxi-0.32*yrange, mini+0.35*yrange), xytext=(mini+0.52*yrange, mini+0.48*yrange), arrowprops=dict(arrowstyle="->", lw=3, color='gray', alpha=0.3))  # royalblue
    axis.set_xlabel(''), axis.set_ylabel('')
    axis.set_title(descript3)

fig.text(0.5, 0.105, 'Young Fraction for the 5-layer variable K model', ha='center', fontsize=12)
fig.text(0.07, 0.48, 'Young Fraction for the 5-layer zoned K model', va='center', fontsize=12, rotation='vertical')
plt.subplots_adjust(top = .86, bottom = 0.15)
fig.suptitle('Comparison of Young Fraction by HUC scale, as informed by model \n'
         'property:  %change in {} of {}'.format(metric, fluxtype), fontsize=16)
dst = '5LhetN_YFCompare_by_%del_{}_of_{}_of_{}.png'.format(metric, fluxtype, mod_factor3)
dst_pth = os.path.join(fig_dir, dst)
f = plt.gcf()
f.savefig(dst_pth)

# Add an attribute to the HUC shapefile
#shapefile = HUCshpdict[each]
#path = os.path.dirname(HUCshpdict[each])
#shp = gpd.read_file(shapefile)  
#shp[each] = shp[each].astype('int64')  # convert so can merge

#ddd2.index = ddd2.index.astype('int64')  # convert so can merge
#df = pd.merge(shp, ddd2, how='outer', left_on=each, right_index=True)
#dst = os.path.join(path, '{}_YF_{}_{}.shp'.format(each, fluxtype, metric))
#df.to_file(dst)